Adapted from the CSV Reports Download notebook

This notebook takes a user_id, downloads a list of courses that user is attached to, downloads a list of all the quizzes in each of those courses, and downlaods a list of all the quiz submissions for each of those quizzes.  No file is downloaded if there are no quizzes or there are no quiz submissions.

Courses: https://canvas.instructure.com/doc/api/courses.html

Quizzes: https://canvas.instructure.com/doc/api/quizzes.html

Quiz Submissions: https://canvas.instructure.com/doc/api/quiz_submissions.html

In [ ]:
#!/usr/bin/env python
# working as of 6/8/2018
import requests
import time, json, os, shutil
import re,pprint
import smtplib
import pandas as pd

### NOTES: ###
# 1. You can use an external credentials file to (slightly) increase security.
#   a. This works for both token and email credential information.
# 2. All <> angle brackets are designed to be replaced with your data. DO NOT INCLUDE THEM.

# Change this to match your access token
# Token in LastPass
from config import token

# This headers dictionary is used for almost every request
headers = {"Authorization":"Bearer %s" % token}

# Change this to match the domain you use to access Canvas
CANVAS_DOMAIN  = "sltr.instructure.com"

# Change this to the full path of your desired output folder - always use / whether linux or Windows
# Always include trailing / and uncomment options to create directories
OUTPUT_FOLDER = "./data/quizzes/"

#if not os.path.exists(OUTPUT_FOLDER):
#   os.makedirs(OUTPUT_FOLDER)

# Canvase links courses to user, so you need to feed a user_id to the API.
# This is easiest to do if you have a master user who is linked to every course.
MASTER_USER_ID = ""

quiz_files = []
submissions_files = []

# Download Quiz data

In [ ]:
# REMOVE OLD FILE(S) FIRST
folder = OUTPUT_FOLDER

try:
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
except FileNotFoundError:
    os.mkdir(folder)

BASE_DOMAIN = "https://%s/api/v1/%%s/" % CANVAS_DOMAIN
COURSE_LIST_URI = BASE_DOMAIN % "users/%s/courses" % MASTER_USER_ID

course_response = requests.get(COURSE_LIST_URI, headers=headers)
course_list = course_response.json()

while "next" in course_response.links:
    next_page_uri = course_response.links["next"]["url"]
    course_response = requests.get(next_page_uri, headers=headers)
    course_list.extend(course_response.json())

pd.DataFrame(course_list, dtype=str).to_csv(f'{folder}course_list.csv', index=False)
course_ids = []
for course in course_list:
    course_ids.append(course['id'])


for course_id in course_ids:
    ###################################################################################
    ############# BE EXTREMELY CAREFUL CHANGING ANY INFORMATION BELOW #################
    QUIZ_LIST_URI = BASE_DOMAIN % "courses/%s/quizzes" % course_id
    QUIZ_DATA_URI = BASE_DOMAIN % "courses/%s/quizzes/%%s/submissions" % course_id
    
    quiz_response = requests.get(QUIZ_LIST_URI, headers=headers)
    quiz_list = quiz_response.json()
    
    while "next" in quiz_response.links:
        next_page_uri = quiz_response.links["next"]["url"]
        quiz_response = requests.get(next_page_uri, headers=headers)
        quiz_list.extend(quiz_response.json())
    
    quiz_file = f"course_{course_id}_quiz_list.csv"
    quiz_df = pd.DataFrame(quiz_list, dtype=str)
    quiz_df['course_id'] = course_id
    quiz_df.to_csv(f'{folder}{quiz_file}', index=False)
    quiz_files.append(f'{folder}{quiz_file}')
    
    for quiz in quiz_list:
        quiz_id = quiz['id']
        quiz_submission_uri = QUIZ_DATA_URI % quiz_id
        
        file_name = f"course_{course_id}_quiz_{quiz_id}.csv"
                
        # NOW REQUEST AND WRITE NEW FILES
        quiz_submission_response = requests.get(quiz_submission_uri, headers=headers)
        quiz_submissions_df = pd.DataFrame(quiz_submission_response.json()['quiz_submissions'], dtype=str)
        
        while "next" in quiz_submission_response.links:
            next_page_uri = quiz_submission_response.links["next"]["url"]
            quiz_submission_response = requests.get(next_page_uri, headers=headers)
            new_data_df = pd.DataFrame(quiz_submission_response.json()['quiz_submissions'], dtype=str)
            quiz_submissions_df = quiz_submissions_df.append(new_data_df, ignore_index=True)
        
        if quiz_submissions_df.size > 0:
            quiz_submissions_df.to_csv(f'{folder}{file_name}', index=False)
            submissions_files.append(f'{folder}{file_name}')